<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [3]:
import logging
import os
import sys

module_path = os.path.abspath(os.path.join('../python/python_src_preprocessing/src'))

if module_path not in sys.path:
    sys.path.append(module_path)

from utils.reader.densitymap import ingredient as density_ingredient, load_directory
from rf.regression import ingredient as rf_ingredient, multiple_forests as multiple

from sacred.experiment import Experiment
from sacred.observers.file_storage import FileStorageObserver

%load_ext autoreload
%autoreload 1
%aimport utils.reader.densitymap, rf.regression

%aimport utils.reader.densitymap

#logging.basicConfig(level=logging.INFO)

simulation = 'check/csvs/obstacle/'
experiment = 'check/csvs/experiment/'

ex = Experiment('my_experiment', ingredients=[density_ingredient, rf_ingredient], interactive=True)

file_storage = FileStorageObserver.create('check/runs', template=os.path.abspath('check/runs/template.txt'))
ex.observers.append(file_storage)

logger = logging.getLogger('ex_logger')

ch = logging.StreamHandler()
formatter = logging.Formatter('%(levelname)s - %(name)s - %(message)s')
ch.setFormatter(formatter)
logger.handlers = []
logger.addHandler(ch)
logger.setLevel(logging.INFO)

ex.logger = logger

@ex.config
def config():
    number_of_targets = 2
    number_of_trees = 50
    save = False

@ex.main
def main(_run, _log, _config):
    print(_config)
    # ex.logger.handlers = []
    
    file_handler = logging.FileHandler(os.path.join(file_storage.dir, 'run.log'))
    file_handler.setFormatter(formatter)
    ex.logger.addHandler(file_handler)
    
    
    _log.info(_run.config)
    
    # load training (simulation data)
    train = load_directory(simulation, _config['number_of_targets'], file_filter=lambda x: 'reduced' in x)

    # load test (experiment data)
    test = load_directory(experiment, _config['number_of_targets'], file_filter=lambda x: 'filtered' in x)
    
    # run
    regressors, \
    normed_prediction, \
    (score_training, score_test, score_oob), \
    errors = multiple( \
        {'samples': train[0], 'targets': train[1]}, \
        {'samples': test[0], 'targets': test[1]}, \
    number_of_targets=_config['number_of_targets'], \
    number_of_trees=_config['number_of_trees'], \
    number_of_cores=4, \
    save=_config['save'])
    
    
    ex.logger.removeHandler(file_handler)
    logger.removeHandler(ch)
    
result = ex.run()

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


INFO - ex_logger.my_experiment - Running command 'main'
INFO - ex_logger.my_experiment - Started run with ID "2"


{'number_of_targets': 2, 'number_of_trees': 50, 'save': False, 'seed': 421394921, 'reader': {'densitymap': {'seperator': ';', 'file_filter': <function <lambda> at 0x000001943DC8B678>}}, 'randomforest': {'regression': {}}}


INFO - ex_logger.main - {'number_of_targets': 2, 'number_of_trees': 50, 'save': False, 'seed': 421394921, 'reader': {'densitymap': {'seperator': ';', 'file_filter': <function <lambda> at 0x000001943DC8B678>}}}
INFO - ex_logger.reader.densitymap.load_file - File check/csvs/obstacle/ORG-KO-240-050-240_2019-06-10_19-37-23.864_filtered_reduced.csv loaded, 213 samples, 240 features per sample, 2 targets per sample
INFO - ex_logger.reader.densitymap.load_file - File check/csvs/obstacle/ORG-KO-240-050-240_2019-06-28_19-17-42.661_filtered_reduced.csv loaded, 219 samples, 240 features per sample, 2 targets per sample
INFO - ex_logger.reader.densitymap.load_file - File check/csvs/obstacle/ORG-KO-240-050-240_2019-06-28_19-26-52.509_filtered_reduced.csv loaded, 224 samples, 240 features per sample, 2 targets per sample
INFO - ex_logger.reader.densitymap.load_file - File check/csvs/obstacle/ORG-KO-240-060-240_2019-06-10_19-38-06.167_filtered_reduced.csv loaded, 202 samples, 240 features per sample,

In [ ]:
# imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as matplot
import itertools
import math
import sys
import os

module_path = os.path.abspath(os.path.join('../python/python_src_preprocessing/src'))

if module_path not in sys.path:
    sys.path.append(module_path)

import utils
    
from rf.regression import multiple_forests as multiple

from utils.reader.densitymap import load_directory
from tqdm import tqdm_notebook, tqdm
from ipywidgets import *

%load_ext autoreload
%autoreload 1
%aimport utils.reader.densitymap, rf.regression
%matplotlib notebook

In [ ]:
# setup
simulation = 'check/csvs/obstacle/'
experiment = 'check/csvs/experiment/'

number_of_targets = 3

In [ ]:
# load training (simulation data)
train = load_directory(simulation, number_of_targets, file_filter=lambda x: 'reduced' in x)

# load test (experiment data)
test = load_directory(experiment, number_of_targets, file_filter=lambda x: 'filtered' in x)

In [ ]:
display(train[1])

In [ ]:
# run
regressors, \
normed_prediction, \
(score_training, score_test, score_oob), \
errors = multiple( \
    {'samples': train[0], 'targets': train[1]}, \
    {'samples': test[0], 'targets': test[1]}, \
    number_of_trees=50)

In [ ]:
print(result.info)